In [38]:
import pandas as pd
import rasterio
import xarray as xr
import numpy as np
import geopandas as gpd

In [2]:
DATA_IN = '/home/CHIRTS/daily/' # <<--- path to loop through
DATA_OUT = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-HI/'
DATA_INTERIM = '/home/cascade/projects/UrbanHeat/data/interim/'

In [3]:
polyRst_fn = 'GHS_UCDB_Raster_touched.tif'
polyRst = rasterio.open(DATA_INTERIM+polyRst_fn)

In [7]:
# Turn polyRst data as Xarray, 
polyRst_da = xr.DataArray(polyRst.read(1), dims = ['y', 'x'])

In [8]:
# Open CHIRT Data and turn data into array
tempRst = rasterio.open(DATA_IN+'1983/HeatIndex.1983.01.01.tif')

In [9]:
# Make arrays into x    array DataArray
tempRst_da = xr.DataArray(tempRst.read(1), dims = ['y', 'x']) # y and x are our 2-d label

In [44]:
# Make xarray dataset
ds = xr.Dataset(data_vars = 
        {'ghs' : (['y', 'x'], polyRst_da),
        'temp' : (['y', 'x'], tempRst_da),})

In [45]:
# Mask pixels for both ghs and chirts where ghs cities are not present
ds_mask = ds.where(ds.temp != -9999, drop = False) #<<<<------ need to double check this
ds_mask = ds_mask.where(ds_mask.ghs > 0, drop = False)
ds_mask - ds_maks.where(ds_mask.temp)

In [46]:
ds_mask

<xarray.Dataset>
Dimensions:  (x: 7200, y: 2600)
Dimensions without coordinates: x, y
Data variables:
    ghs      (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan
    temp     (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan

In [21]:
# Group poly_IDs find temp
avg = ds_mask.groupby('ghs').mean(xr.ALL_DIMS)

/home/cascade/miniconda3/envs/geo/lib/python3.6/site-packages/xarray/core/nanops.py:160: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [22]:
avg_ID = np.array(avg.ghs)
avg_temp = np.array(avg.temp)

In [33]:
df_avg = pd.DataFrame()
df_avg['1983.07.01'] = avg_temp
df_avg['ID_HDC_G0'] = ([int(x) for x in avg_ID])


In [34]:
df_avg

,1983.07.01,ID_HDC_G0
0,77.237808,1
1,NaN,2
2,54.081848,3
3,57.051075,4
4,53.076546,5
...,...,...
13067,60.936974,13131
13068,90.383125,13132
13069,87.268105,13133
13070,87.334717,13134


In [36]:
df_nan = df_avg[df_avg['1983.07.01'].isnull()]

In [39]:
ghs = gpd.read_file('/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/raw/GHS_UCDB/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp')


In [40]:
out = ghs.merge(df_nan, on = 'ID_HDC_G0', how = 'right')

In [43]:
#out.to_file('~/projects/')